In [1]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg

camera = Camera.instance(width=720, height=720) # 720x720 픽셀 이미지 사용

In [2]:
import os

os.system("sudo chmod 777 /dev/ttyTHS1")

256

In [4]:
from servoserial import ServoSerial
import time

pantilt = ServoSerial()

pantilt.Servo_serial_control(2, 3600) # 36 중앙값
time.sleep(5)

serial Open!
161
b'\xff\xff\x02\x07\x03*\x0e\x10\x00\n\xa1'


In [5]:
global face_x, face_y, face_w, face_h
face_x = face_y = face_w = face_h = 0


In [6]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

face_image = widgets.Image(format='jpeg', width=300, height=300) # 프리뷰 화면은 300x300 픽셀 크기로
display(face_image)

Image(value=b'', format='jpeg', height='300', width='300')

In [7]:
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [ ]:
while 1:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300)) # 이미지를 학습 데이터와 동일한 크기로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 흑백 이미지로 변환하여 검출에 사용
    
    faces = face_cascade.detectMultiScale(gray) # 이미지 내에서 얼굴을 검출
    
    if len(faces) > 0: # 검출된 내용이 있으면
        (face_x, face_y, face_w, face_h) = faces[0] # 좌표와  크기
        
        # 검출 영역을 사각 박스로 표시합니다.
        cv2.rectangle(frame, (face_x, face_y), (face_x+face_h, face_y+face_w), (0,255,0), 4)
        
        # 눈 검출은 얼굴이 검출된 영역 내부에서만 진행하기 위해 ROI를 생성합니다.
        roi_gray = gray[face_y:face_y+face_h, face_x:face_x+face_w]
        roi_color = frame[face_y:face_y+face_h, face_x:face_x+face_w]
        
        # 얼굴 안에서 눈을 검출합니다.
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        # 눈이 검출되었다면 눈 위치에 대한 좌표 정보를 리턴받습니다.
        for (ex, ey, ew, eh) in eyes:
            
            # 원본 이미지에 눈의 위치를 표시합니다. (roi_color에 표시하면 원본 이미지에도 표시용)
            cv2.rectangle(roi_color, (ex,ey), (ex+ew, ey+eh), (255,0,0), 2)
            
    # 결과 이미지를 프리뷰 위젯에 표시
    face_image.value = bgr8_to_jpeg(frame)
    
    # 눈 검출은 얼굴이 검출된 영역 내부에서만 진행하기 위해 ROI를 생성합니다.
    roi_gray = gray[face_y:face_y+face_h, face_x:face_x+face_w]
    roi_color = frame[face_y:face_y+face_h, face_x:face_x+face_w]
    

In [136]:
# 팬/틸트 각도를 제어하는 함수
def MovePanTilt(pan, tilt):
    pantilt.Servo_serial_double_control(1, pan, 2, tilt)
    

In [141]:
# MID_TILT = 3600
# TILT 범위 3000~4095
MID_PAN = 4095 // 2 + 100

DEF_TILT = 4000

In [142]:
# 위 아래로 흔들기 3번 반복
for i in range(0, 3):
    MovePanTilt(MID_PAN, DEF_TILT+300)
    time.sleep(0.2)
    MovePanTilt(MID_PAN, DEF_TILT-300)
    time.sleep(0.2)
    
MovePanTilt(MID_PAN, 4000) # 기본 위치
time.sleep(1)

178
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xff\x00\n\xb2'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0et\x00\n>'
178
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xff\x00\n\xb2'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0et\x00\n>'
178
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xff\x00\n\xb2'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0et\x00\n>'
17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'


In [77]:
# 좌우로 흔들기 3번 반복
for i in range(0, 3):
    MovePanTilt(MID_PAN+300, DEF_TILT)
    time.sleep(0.2)
    MovePanTilt(MID_PAN-300, DEF_TILT)
    time.sleep(0.2)
    
MovePanTilt(MID_PAN, DEF_TILT) # 기본 위치
time.sleep(1)

228
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x8f\x00\n\x02\x0f\xa0\x00\n\xe4'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x077\x00\n\x02\x0f\xa0\x00\n>'
228
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x8f\x00\n\x02\x0f\xa0\x00\n\xe4'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x077\x00\n\x02\x0f\xa0\x00\n>'
228
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x8f\x00\n\x02\x0f\xa0\x00\n\xe4'
62
b'\xff\xff\xfe\x0e\x83*\x04\x01\x077\x00\n\x02\x0f\xa0\x00\n>'
17
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08c\x00\n\x02\x0f\xa0\x00\n\x11'
